# Enterprise Steam for manging AI engines

This notebook is intended to help you get started managing your AI Engines in the H2O AI Cloud using python. 

* **Product Documentation:** https://docs.h2o.ai/steam/index.html
* **Python Documentation:** https://docs.h2o.ai/enterprise-steam/latest-stable/docs/python-docs/index.html

## Prerequistes

Update the `h2o_ai_cloud.py` file with the connection parameters for your H2O AI Cloud environemnt:
1. Login to your H2O AI Cloud environment
1. Click your username or avatar in the H2O AI Cloud navigation bar
1. Navigate to `CLI & API Access`
1. Use the variables from the `Accessing H2O AI Cloud APIs` section to populate the parameters

In [1]:
from getpass import getpass

from h2o_ai_cloud import steam_client
from h2osteam.clients import DriverlessClient, H2oKubernetesClient

import h2osteam

## Securely connect

We first connect to the H2O AI Cloud using our platform token to create a token provider object. We can then use this object to log into Steam and other APIs.

In [35]:
steam = steam_client()

Visit https://cloud.h2o.ai/auth/get-platform-token to get your platform token
Enter your platform token: ········


## Understanding your environment

* Ensure that your python client and backend versions match. 
* View what AI Engines and environments are available 

In [3]:
print(f"Backend Version: {h2osteam.api().get_config_meta()['version']}")
print(f"Library Version: {h2osteam.__version__}")

Backend Version: 1.8.12
Library Version: 1.8.12


In [4]:
h2osteam.api().get_config_meta()

{'version': '1.8.12',
 'build': '1.8.11-14-g45e554b-dirty',
 'built': 'Wed May 11 22:14:52 UTC 2022',
 'restart_pending': True,
 'support_email': 'cloud-feedback@h2o.ai',
 'license_valid': True,
 'is_hadoop_enabled': False,
 'is_kubernetes_enabled': True,
 'is_h2o_enabled': True,
 'is_h2o_running': True,
 'is_sparkling_enabled': False,
 'is_sparkling_running': False,
 'is_driverless_enabled': True,
 'is_driverless_running': True,
 'is_h2o_engine_uploaded': False,
 'is_h2o_kubernetes_engine_uploaded': True,
 'is_sparkling_engine_uploaded': False,
 'is_driverless_engine_uploaded': True,
 'driverless_backend_type': 'kubernetes',
 'is_minio_enabled': False,
 'h2o_backend_type': 'kubernetes',
 'inside_cluster': True}

## Managing Driverless AI

### View available versions

In [5]:
for dai_version in steam.get_driverless_engines():
    print(dai_version["version"])

1.10.1.3
1.10.2
1.10.3


### View available profiles

* List the profiles
* View the options for max run time in this profile

In [36]:
dai_profiles = [profile["name"] for profile in steam.get_profiles() if profile["profile_type"] == "driverless_kubernetes"]

print(dai_profiles)

['visitor-driverless-kubernetes']


In [7]:
steam.get_profile_by_name(dai_profiles[0])["driverless_kubernetes"]["max_uptime_hours"]

{'id': 1005,
 'min': 1,
 'is_min_set': True,
 'max': 24,
 'is_max_set': True,
 'initial': 8,
 'is_initial_set': True,
 'profile_max': -1,
 'is_profile_max_set': True}

### Create an instance
See the `3 AutoML` tutorial for what to do once you've connected to the AutoML platform. 

In [8]:
dai_machine = DriverlessClient(steam).launch_instance(
    name="my-test-instance",
    version=steam.get_driverless_engines()[-1]["version"],  # newest version of the AutoML software,
    profile_name=dai_profiles[0],
    max_uptime_h=1
)
client = dai_machine.connect()

Driverless AI instance is submitted, please wait...
Driverless AI instance is running


In [9]:
dai_machine.details()

{'id': 3283,
 'profile_name': 'default-driverless-kubernetes',
 'name': 'my-test-instance',
 'status': 'running',
 'target_status': 'running',
 'version': '1.10.3',
 'backend_type': '',
 'instance_type': 'singlenode',
 'master_id': -1,
 'cpu_count': 8,
 'gpu_count': 0,
 'memory_gb': 30,
 'storage_gb': 64,
 'max_idle_seconds': 7200,
 'max_uptime_seconds': 3600,
 'timeout_seconds': 1800,
 'address': 'http://10.1.212.19:12345',
 'authentication': 'oidc',
 'password': '',
 'created_at': 1657064256,
 'started_at': 1657064304,
 'created_by': 'michelle.tanco@h2o.ai',
 'current_uptime_seconds': 11,
 'current_idle_seconds': 11,
 'pod_latest_event': None,
 'service_latest_event': None,
 'pvc_latest_event': None,
 'stop_reason': '',
 'config_toml': '',
 'volumes': ''}

In [10]:
dai_machine.openid_login_url()

'https://steam.cloud.h2o.ai:443/oidc-login-start?forward=/proxy/driverless/3283/openid/callback'

### List my instances

In [37]:
for instance in steam.get_driverless_instances():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

3274 	 visitor-driverless-kubernetes 	 stopped 	 dai-quickstart
3279 	 visitor-driverless-kubernetes 	 stopped 	 my-test-instance


### Download instance logs

In [12]:
dai_machine = DriverlessClient(steam).get_instance(name="my-test-instance")

In [13]:
dai_machine.download_logs('dai_test_instance_logs.zip')

Driverless AI instance logs saved to dai_test_instance_logs.zip


In [14]:
!unzip dai_test_instance_logs.zip -d dai_test_instance_logs

Archive:  dai_test_instance_logs.zip
replace dai_test_instance_logs/steam-config.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


### Pause an instance

You can pause an instance that is currently running. Pausing an instance shuts it down to save resources, it is similar to powering off a server. You will not loose any data and you can start an instance at any time.

In [15]:
dai_machine.stop()

Driverless AI instance is stopping, please wait...
Driverless AI instance is stopped


### Resume the instance

You can resume a paused instance by running:

In [16]:
dai_machine.start()

Driverless AI instance is starting, please wait...
Driverless AI instance is running


### Delete the instance

When you no longer need an instance, you can terminate it. Once deleted, there is no way to restart the instance or access any data.

In [17]:
dai_machine.terminate()

In [38]:
# it takes ~20 seconds for the status to update
for instance in steam.get_driverless_instances():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

3274 	 visitor-driverless-kubernetes 	 stopped 	 dai-quickstart
3279 	 visitor-driverless-kubernetes 	 stopped 	 my-test-instance


## Managing H2O-3

### View available versions

In [19]:
for h2o_version in steam.get_h2o_kubernetes_engines():
    print(h2o_version["version"])

3.34.0.7
3.36.0.1
3.36.0.4


### View available profiles

In [39]:
h2o_profiles = [profile["name"] for profile in steam.get_profiles() if profile["profile_type"] == "h2o_kubernetes"]

print(h2o_profiles)

['visitor-h2o-kubernetes']


In [21]:
steam.get_profile_by_name(h2o_profiles[0])["h2o_kubernetes"]["max_uptime_hours"]

{'id': 1026,
 'min': 1,
 'is_min_set': True,
 'max': 24,
 'is_max_set': True,
 'initial': 12,
 'is_initial_set': True,
 'profile_max': -1,
 'is_profile_max_set': True}

### Create a cluster
See the H2O-3 Tutorial for what to do once you've connected to the distributed ML platform. 

In [22]:
h2o_machine = H2oKubernetesClient(steam).launch_cluster(
    name="my-test-instance",
    version=steam.get_h2o_kubernetes_engines()[-1]["version"],  # newest version of the AutoML software,
    profile_name=h2o_profiles[0],
    max_uptime_h=1
)

In [23]:
while not h2o_machine.is_running():
    pass

In [24]:
client = h2o_machine.connect()

Connecting to H2O server at https://steam.cloud.h2o.ai:443/proxy/h2o-k8s/945 ... successful.


H2O_cluster_uptime:,18 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,3 months and 5 days
H2O_cluster_name:,my-test-instance
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,817 Mb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


In [25]:
print("https://" + h2o_machine.get_connection_config()["connect_params"]["ip"] + "/" + h2o_machine.get_connection_config()["connect_params"]["context_path"] + "/flow/index.html")

https://steam.cloud.h2o.ai/proxy/h2o-k8s/945/flow/index.html


### List my clusters

In [40]:
for instance in steam.get_h2o_kubernetes_clusters():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

### Download cluster logs

In [27]:
h2o_machine = H2oKubernetesClient().get_cluster(name="my-test-instance", created_by="")

In [28]:
h2o_machine.download_logs('h2o_test_instance_logs.zip')

H2O cluster logs saved to h2o_test_instance_logs.zip


In [29]:
!unzip h2o_test_instance_logs.zip -d h2o_test_instance_logs

Archive:  h2o_test_instance_logs.zip
  inflating: h2o_test_instance_logs/server.log  
  inflating: h2o_test_instance_logs/events.log  
  inflating: h2o_test_instance_logs/h2o.log  
  inflating: h2o_test_instance_logs/steam-config.yaml  


### Stop an instance
H2O clusters cannot be resumed after they are stopped. Only turn off your cluster once you are permanately done with it.

In [30]:
h2o_machine.stop()

In [31]:
print(f"Current status: {h2o_machine.status}")
print(f"Target status: {h2o_machine.target_status}")

Current status: running
Target status: stopped


### Delete an instance

In [32]:
h2o_machine.terminate()

In [41]:
for instance in steam.get_h2o_kubernetes_clusters():
    print(instance["id"], "\t", instance["profile_name"], "\t", instance["status"], "\t", instance["name"])

## Clean up

In [34]:
!rm dai_test_instance_logs.zip
!rm -rf dai_test_instance_logs

!rm h2o_test_instance_logs.zip
!rm -rf h2o_test_instance_logs